# Machine Learning For Lithography
## Unit II : Linear Classifiers

### Introduction

The Linear Classifier is a simple model form and a useful introduction for neural networks.  It also has, available from the Google TensorFlow team, a "Canned Estimator" which means that the model for mahs been already assembled for our use in TensorFlow.

Suppose the network is trained with a dataset D consisting of training input data samples X and labels y(X).  

Further suppose that acceptable values of y are in a finite set of N different classifications C={c1, c2, ...cN}.

In the script we  will call the number of different classifications "n_classes."

The number of features per input sample X is 48x48 = 2304.

The Linear Classifier model will effectively build a linear model for each class, predicting the confidence that the input sample belongs in each class. 

To interpret the confidence as a probability, and simplify the calculation of the cost function, the softmax operation will normalize the sum over the probability distribution of all of the classes.

### Preamble
This section imports some necessary packages and helper functions that enable our script.

Of particular importance is TensorFlow, here imported as "tf," which is the nickname by which we will be able to access it in our script.  TensorFlow is our machine learning framework, enabling definition of the model form, definition of the training and validation procedures, definition of the model prediction method, and implementation of the training and prediction procedures.

We also import numpy, which we will reference with the nickname "np".  The name "numpy" is short for "numerical python".  The numpy package is a critical cornerstone of the data science workflow, providing intuitivce and interactive support for arrays in a fashion that will be familiar to those who have previously done work in matlab.

The matplotlib library is a nice set of tools for looking at our aerial images.

The methods loaded from "classes" are little helper functions I wrote to make the demo script you see more compact and focused on Machine Learning rather than munging data and logs and visualizations.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from classes.Visualizations import *
from classes.Data import  loadResNIST
from classes.Specs import specs
import logging


This part of the preamble just sets some useful variables that help keep our log data separate from the other model forms.

In [ ]:
DATADIR='./resNIST/'
LOGDIR = './linear_classifier_logs/'
PROJECTORDIR=LOGDIR+'projector/'
summary_writer = tf.summary.FileWriter(LOGDIR)
scopes = ['linear/linear_model/linear_model/weighted_sum:0','linear/head/labels:0']

### Load and Transform Data

We introduce here the "tf.estimator" which is a facility from tensorflow that helps standardize the definition of model training, validation and predicition pipelines.

In this code block we are loading our data into four blocks:
1. **train_data** : the input training data **X**, representing a set of samples of aerial images, each 48x48 pixels. 
2. **train_labels** : the label **y(X)**, belonging to one of 11 classes, **c in C={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}**.  These class labels are integers, but they represent the proportion of the pixel vicinity that is "covered" by photoresist after development. A 0 denotes "not covered." A 1 denotes "fully or 100% covered."  Each increase in the index of the label correspondes to in increase in resist coverage of 10%.
3. **eval_data** : these samples **X** are held out from training so that we may evaluate the variance and detect potential overfitting.
4. **eval_labels** : these labels are sued in conjunction with **eval_data** to help detect overfitting.

The code then defines two input functions, one for training (**train_input_fn**) and one for evaluation (**eval_input_fn**), according to the "numpy_input_fn" spec which helps facilitate feeding tensorflow batches of samples.  We indicate that the training input function will be fed from the **train_data** and **train_labels** variables, and likewise the evaluation input function will be fed from the **eval_data** and **eval_labels** variables.

For training we specify a mini-batch size, which determines how how many samples are averaged together in determining an update direction for adjusting the weights.  

During training we shuffle the dataset before breaking it into mini-batches, to prevent correlations from data preparation from skewing results or avoid reliance on lucky fits.  However, to ensure consistency when evaluating the data during training we do not shuffle during evaluation.

Finally, we take the 48x48 image for each input sample and break it into a long 2304 row, with each pixel belonging to its own "feature column" for every image.  This feature_column will be the front end of our TensorFlow model.

In [ ]:
image_size=48
train_data, train_labels , eval_data, eval_labels    = loadResNIST(DATADIR)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=32,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
feature_columns=[tf.feature_column.numeric_column("x",shape=[image_size,image_size])]

### Define the Estimator
We use the RunConfig facility of the tf.estimator to hspecify how frequently we want to checkpoint the model (save intermediate results.). This also influences how frequently we will perform validation.

Because we are using a LinearClassifier canned estimator, it is very easy to define the model form.  It is actually very commonly the case that you can begin a machine learning project in tensorflow by adapting an existing model form, rather than writing it from scratch.  There are many examples available on the web, including this demo from the TensorFlow team which influenced my writing of this script:
https://www.tensorflow.org/tutorials/estimators/linear

We are defining 11 **n_classes** because we are binning resist coverage into tenths, including the terminal conditions of 0% coverage and 100% coverage.

We offer an example of how to control the choice of the optimizer, which is "Follow the Regularized Leader" or FTRL by default.  GradientDescent is a more common estimator for neural networks but FTRL works well for this case, particularly for large numbers of features.  We show how you can influence the strength of the L1 regularization (which encourages weight sparsity) as well as L2 shrinkage regularization (which encourages small weights and helps prevent overfitting such as that which can result from subtractions of large numbers.)

In [ ]:
config=tf.estimator.RunConfig(save_checkpoints_steps=1000)

linear_classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    config=config,
    n_classes=11,
    optimizer=tf.train.FtrlOptimizer(learning_rate=0.1,
                                    l1_regularization_strength=0.02,
                                    l2_shrinkage_regularization_strength=0.02),
#    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01),
    model_dir=LOGDIR
)

We plan on using the "train_and_eval" method provided for the tf.estimator class, because it automates a periodic evaluation of the model during training, generating occasional checkpoints and then loading those checkpoints in to assess the model performance on the evaluation data.  In order to do this we need to activate the "logger" that logs data, and we need to define the **train_spec** and **eval_spec** which specify some of the details of the process, including directories for logged data, duration of the training process and frequency of logged data.

The function "specs" is a little helper function I wrote for added compactness of this lab exercise.

In [ ]:
logging.getLogger().setLevel(logging.INFO)  
# to show info about training progress in the terminal
train_spec, eval_spec = specs(train_input_fn, eval_input_fn, 
                              logdir=LOGDIR, projectordir=PROJECTORDIR, 
                              max_train_steps=500000, eval_steps = 1000, 
                              scopes = scopes, name = 'lc')

In this elegant line of code we ask tensorflow to begin the training process, with periodic evaluation, using the linear_classifier model and the training and eval specs we previously defined. Nice and compact!

In [ ]:
tf.estimator.train_and_evaluate(linear_classifier, train_spec, eval_spec)

This is how we can evaluate the accuracy of the model independent of the training data.

In [ ]:
accuracy_score = linear_classifier.evaluate(input_fn=eval_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

This code prepares the data and metadata for plotting in TensorBoard using the Principal Components Analysis (PCA) and t-SNE projection methods for visualizing in high-dimensions.  The prepare_projector and prepare_sprites functions are little tidy script I wrote to simplify the lab.

In [ ]:
prepare_projector(PROJECTORDIR, scopes)
prepare_sprites(PROJECTORDIR, eval_data)

We can see a specific example by picking an index and using the "matplotlib" library to make a nice picture.

In [ ]:
plt.imshow(eval_data[4117])